In [1]:
#imports
import pandas as pd 
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time
import matplotlib.pyplot as plt
import math
import numpy as np
import datetime
import json
from dateutil import parser
from datetime import datetime, timezone
import nba_api
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import playercareerstats, leaguegamefinder, playerdashboardbyclutch, playergamelogs, commonplayerinfo, teamplayeronoffdetails, teamgamelogs
from nba_api.live.nba.endpoints import scoreboard

In [2]:
class Team(object):
    def __init__(self, team_abbreviation, teamId):
        
        
        self.team_abbreviation = team_abbreviation
        self.TeamID =  teamId
        self.TeamName = teams.find_team_name_by_id(team_id=self.TeamID)['full_name']
        self.StartYear = 2020 #First year data is avalible
        self.LastYear = 2021 #Current year
        
        
    #Complile the Team Data
    def compile_active_team_data(self):
    
        team_season_point_average_raw_data = []
        team_points_raw_data = []
        opponent_season_point_average_raw_data = []
        opponent_season_points_raw_data = []
        
        team_season_FT_average_raw_data = []
        team_season_FG_average_raw_data = []
        team_season_FG3_average_raw_data = []
        opponent_season_FT_average_raw_data = []
        opponent_season_FG_average_raw_data = []
        opponent_season_FG3_average_raw_data = []
            
        
        print(self.StartYear, self.LastYear)
        for year in range((self.StartYear), (self.LastYear)):
            print("-------------------------------------------------")
            season = str(year) + "-" + str(((year+1)-2000)) #format the season correctly for the API
            teamLogs = teamgamelogs.TeamGameLogs(team_id_nullable=self.TeamID, season_nullable=season) #Get the TeamLogs for the season
            print(season)
            

            matchup_api_data = teamLogs.get_data_frames()[0]["MATCHUP"]
            team_season_points_api_data=teamLogs.get_data_frames()[0]["PTS"]
            self.team_abbreviation = teamLogs.get_data_frames()[0]["TEAM_ABBREVIATION"][0]
            

            team_season_points = []
            for game in range(0, len(teamLogs.get_data_frames()[0])):
                   
                    #Isolate the opponent abbreviation
                    try:   
                        matchup_api_data[game] = matchup_api_data[game].replace(self.team_abbreviation, "")
                    except:
                        print("Unexpected Error with removing player team")
                    try:
                        matchup_api_data[game] = matchup_api_data[game].replace(" @ ", "")
                        matchup_api_data[game] = matchup_api_data[game].replace(" vs. ", "")
                    except:
                        print("Error while isolating the opponent abbreviation")
                    
                    opponentAbbreviation = matchup_api_data[game]
                    
                    #Account for special cases where team names have changed since 2009
                    matchup_api_data[game] = matchup_api_data[game].replace('NOH', 'NOP')
                    matchup_api_data[game] = matchup_api_data[game].replace('NJN', 'BKN')
                    
                    #get the opponent ID
                    opponentAbbreviation_for_search = matchup_api_data[game]    
                    opponentID = int(teams.find_team_by_abbreviation(opponentAbbreviation_for_search)["id"])
                    
                    
                    if (game%10==0):
                        print("processing game {} out of {} for the {} season".format(game, len(teamLogs.get_data_frames()[0]), season))
                        
                    opponentLogs = teamgamelogs.TeamGameLogs(team_id_nullable=opponentID, season_nullable=season) #get Team logs for the opposing team
                    opponent_season_points_api_data = opponentLogs.get_data_frames()[0]['PTS']
                    
                    
                    team_points_raw_data.append(team_season_points_api_data[game])
                    team_season_points.append(team_season_points_api_data[game])
                    
                    past_games = teamLogs.get_data_frames()[0].head(game+1).copy()
                    team_season_FT_average_raw_data.append(past_games.loc[past_games['MATCHUP'].isin(["{} @ {}".format(self.team_abbreviation, opponentAbbreviation),"{} vs. {}".format(self.team_abbreviation, opponentAbbreviation)])]["FTM"].mean())
                    team_season_FG_average_raw_data.append(past_games.loc[past_games['MATCHUP'].isin(["{} @ {}".format(self.team_abbreviation, opponentAbbreviation),"{} vs. {}".format(self.team_abbreviation, opponentAbbreviation)])]["FGM"].mean())
                    team_season_FG3_average_raw_data.append(past_games.loc[past_games['MATCHUP'].isin(["{} @ {}".format(self.team_abbreviation, opponentAbbreviation),"{} vs. {}".format(self.team_abbreviation, opponentAbbreviation)])]["FG3M"].mean())
                    
                    
                    if game == 0:
                        opponent_season_point_average_raw_data.append(opponent_season_points_api_data[game])
                        team_season_point_average_raw_data.append(teamLogs.get_data_frames()[0]["PTS"][game])
                        
                        
                    elif game<(len(opponentLogs.get_data_frames()[0]['PTS'])):
                        opponent_season_point_average_raw_data.append(opponent_season_points_api_data.head(game).mean())
                        team_season_point_average_raw_data.append((sum(team_season_points)/len(team_season_points)))
                                              
                        
                    else:
                        opponent_season_point_average_raw_data.append(opponent_season_points_api_data.mean())
                        team_season_point_average_raw_data.append((sum(team_season_points)/len(team_season_points)))
                    
                                            
                    time.sleep(20)
                    

        #Lengths of lists used for troublshooting in case of error
#         print("the lengths")
#         print(len(team_points_raw_data))
#         print(len(team_season_point_average_raw_data))
#         print(len(opponent_season_point_average_raw_data))
#         print(len(team_season_FT_average_raw_data)) 
#         print(len(team_season_FG_average_raw_data)) 
#         print(len(team_season_FG3_average_raw_data)) 
        
       
        #adding the lists to the team data dictionary to be made into a dataframe
        team_data = {
        "Points Scored": team_points_raw_data,
        "Team Point Average" : team_season_point_average_raw_data,
        "Opponent Team Point Average": opponent_season_point_average_raw_data, 
        "Team Season FTM Average Specific": team_season_FT_average_raw_data,
        "Team Sesason FGM Average Specific": team_season_FG_average_raw_data,
        "Team Season FG3M Average Specific": team_season_FG3_average_raw_data,
        }

        team_dataFrame = pd.DataFrame(data=team_data)
        

        return(team_dataFrame)


       
    #Get the Team data for outside calls
    def get_active_team_data(self):
        return(self.compile_active_team_data())
    def compile_data_for_prediction(self, opponentAbbreviation_for_prediction):
        
        season = "2020-21"
        teamLogs = teamgamelogs.TeamGameLogs(team_id_nullable=self.TeamID, season_nullable=season)
        team_season_points_api_data = teamLogs.get_data_frames()[0]["PTS"]
       
        team_season_point_average = [team_season_points_api_data.mean()]
        
        
        opponentID = int(teams.find_team_by_abbreviation(opponentAbbreviation_for_prediction)["id"])
        opponentLogs = teamgamelogs.TeamGameLogs(team_id_nullable=opponentID, season_nullable=season)
        opponent_season_points_api_data = opponentLogs.get_data_frames()[0]["PTS"]
        opponent_season_point_average = [opponent_season_points_api_data.mean()]

        past_games = teamLogs.get_data_frames()[0].copy()
        
        
        team_season_FT_average = (past_games.loc[past_games['MATCHUP'].isin(["{} @ {}".format(self.team_abbreviation, opponentAbbreviation_for_prediction),"{} vs. {}".format(self.team_abbreviation, opponentAbbreviation_for_prediction)])]["FTM"].mean())
        team_season_FG_average = (past_games.loc[past_games['MATCHUP'].isin(["{} @ {}".format(self.team_abbreviation, opponentAbbreviation_for_prediction),"{} vs. {}".format(self.team_abbreviation, opponentAbbreviation_for_prediction)])]["FGM"].mean())
        team_season_FG3_average = (past_games.loc[past_games['MATCHUP'].isin(["{} @ {}".format(self.team_abbreviation, opponentAbbreviation_for_prediction),"{} vs. {}".format(self.team_abbreviation, opponentAbbreviation_for_prediction)])]["FG3M"].mean())
        
        #If there is not enough data for this season, refer to the last season
        if math.isnan(team_season_FT_average):
            
            teamLogs = teamgamelogs.TeamGameLogs(team_id_nullable=self.TeamID, season_nullable="2019-20")
            past_games = teamLogs.get_data_frames()[0].copy()                                     
            team_season_FT_average = (past_games.loc[past_games['MATCHUP'].isin(["{} @ {}".format(self.team_abbreviation, opponentAbbreviation_for_prediction),"{} vs. {}".format(self.team_abbreviation, opponentAbbreviation_for_prediction)])]["FTM"].mean())
            team_season_FG_average = (past_games.loc[past_games['MATCHUP'].isin(["{} @ {}".format(self.team_abbreviation, opponentAbbreviation_for_prediction),"{} vs. {}".format(self.team_abbreviation, opponentAbbreviation_for_prediction)])]["FGM"].mean())
            team_season_FG3_average = (past_games.loc[past_games['MATCHUP'].isin(["{} @ {}".format(self.team_abbreviation, opponentAbbreviation_for_prediction),"{} vs. {}".format(self.team_abbreviation, opponentAbbreviation_for_prediction)])]["FG3M"].mean())
       
       
        
        prediction_data = {
        "Team Point Average" : team_season_point_average,
        "Opponent Team Point Average": opponent_season_point_average, 
        "Team Season FTM Average Specific": team_season_FT_average,
        "Team Sesason FGM Average Specific": team_season_FG_average,
        "Team Season FG3M Average Specific": team_season_FG3_average
        }
        prediction_dataframe = pd.DataFrame(data=prediction_data)
      #  print(prediction_dataframe)
        return(prediction_dataframe)
        
    def get_data_for_prediction(self, opponentAbbreviation_for_prediction):
        return(self.compile_data_for_prediction(opponentAbbreviation_for_prediction))
        
    



In [3]:
class Prediction(object):
    
    def __init__(self, data):
        self.data = data.copy()
        
    #Normalize the data
    def sigmoid(self, dataframe):
        
        return((dataframe - self.train_stats['mean'])/self.train_stats['std'])
        
    #Defining the model
    def build_model(self):
        
        input_layer=tf.keras.layers.Input(([len(self.train_dataset.keys())]))
        densel_layer = tf.keras.layers.Dense(units=1, input_shape=([len(self.train_dataset.keys())],))
        output = densel_layer(input_layer)
        model = tf.keras.Model(inputs=input_layer,outputs=output)

        model.compile(loss="mse",optimizer=tf.keras.optimizers.Adam(0.01), metrics=['mae', 'mse'])

        return model
        
    def make_prediction(self, prediction_data):
        
        #set up train and testing data
        self.train_dataset = self.data.sample(frac=0.90, random_state=0)
        self.test_dataset = self.data.drop(self.train_dataset.index)
    
        self.train_labels = self.train_dataset.pop("Points Scored")
        self.test_labels = self.test_dataset.pop("Points Scored")
        
        self.train_stats = self.train_dataset.describe()
        #self.train_stats.pop("Points Scored")
        self.train_stats = self.train_stats.transpose()
        
        #normalize the data
        normed_train_data = self.sigmoid(self.train_dataset)
        normed_test_data = self.sigmoid(self.test_dataset)

        #build the model
        model = self.build_model()
     #   model.summary()
        example_batch = normed_train_data
   #     print((example_batch.keys()))
        example_result = model.predict(example_batch)
    #    example_result

        EPOCHS = 1000
        early_stop = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
        history = model.fit(normed_train_data, self.train_labels, epochs=EPOCHS, callbacks=[early_stop])

        #get key metrics
        loss, mae, mse = model.evaluate(normed_test_data, self.test_labels, verbose=0)

   #     print("Testing set Mean Abs Error: {:5.2f} Points Scored".format(mae))
        
        #make predictions and then plot them in realation to the actual values
        test_predictions = model.predict(normed_test_data).flatten()

#         plt.scatter(self.test_labels, test_predictions)
#         plt.xlabel("True Values [Points Scored]")
#         plt.ylabel("Predictions [Points Scored]")
#         plt.axis('equal')
#         plt.axis('square')
#         plt.xlim([60, plt.ylim()[1]])
#         plt.ylim([60, plt.ylim()[1]])
#         _ = plt.plot([-100,200], [-100, 200])

        
        
        #print tests for reference
   #     print(self.test_labels)
   #     print(test_predictions)
        
        #make prediction
        normed_prediction_data = self.sigmoid(prediction_data)
        game_prediction = model.predict(normed_prediction_data).flatten()
        
        return(game_prediction, mae) #return predicted values


In [4]:
board = scoreboard.ScoreBoard()
teamsTonight = []
print("ScoreBoardDate: " + board.score_board_date)
games = board.games.get_dict()
for game in games:
    teamsTonight.append({"teamAbrev" : game['homeTeam']['teamTricode'],"teamId":game['homeTeam']['teamId'] })
    teamsTonight.append({"teamAbrev" : game['awayTeam']['teamTricode'],"teamId":game['awayTeam']['teamId'] })
#teamsTonight = teamsTonight[:2]
teamsTonight

ScoreBoardDate: 2021-11-30


[{'teamAbrev': 'BKN', 'teamId': 1610612751},
 {'teamAbrev': 'NYK', 'teamId': 1610612752},
 {'teamAbrev': 'TOR', 'teamId': 1610612761},
 {'teamAbrev': 'MEM', 'teamId': 1610612763},
 {'teamAbrev': 'PHX', 'teamId': 1610612756},
 {'teamAbrev': 'GSW', 'teamId': 1610612744},
 {'teamAbrev': 'POR', 'teamId': 1610612757},
 {'teamAbrev': 'DET', 'teamId': 1610612765},
 {'teamAbrev': 'SAC', 'teamId': 1610612758},
 {'teamAbrev': 'LAL', 'teamId': 1610612747}]

In [5]:
teamPredData = []
for i in range(0,len(teamsTonight),2):
    homeTeam = Team(teamsTonight[i]['teamAbrev'], teamsTonight[i]['teamId'])
    homeTeamData = homeTeam.get_active_team_data()
    homeTeamPredictions = Prediction(homeTeamData)
    awayTeam = Team(teamsTonight[i+1]['teamAbrev'], teamsTonight[i+1]['teamId'])
    awayTeamData = awayTeam.get_active_team_data()
    awayTeamPredictions = Prediction(awayTeamData)
    teamPredData.append({"homeAbrev":teamsTonight[i]['teamAbrev'],"homeId": teamsTonight[i]['teamId'], "homeTeam": homeTeam, "homeTeamPredictions": homeTeamPredictions,"awayAbrev":teamsTonight[i+1]['teamAbrev'],"awayId": teamsTonight[i+1]['teamId'], "awayTeam": awayTeam, "awayTeamPredictions": awayTeamPredictions})
teamPredData

2020 2021
-------------------------------------------------
2020-21
processing game 0 out of 72 for the 2020-21 season
processing game 10 out of 72 for the 2020-21 season
processing game 20 out of 72 for the 2020-21 season
processing game 30 out of 72 for the 2020-21 season
processing game 40 out of 72 for the 2020-21 season
processing game 50 out of 72 for the 2020-21 season
processing game 60 out of 72 for the 2020-21 season
processing game 70 out of 72 for the 2020-21 season
2020 2021
-------------------------------------------------
2020-21
processing game 0 out of 72 for the 2020-21 season
processing game 10 out of 72 for the 2020-21 season
processing game 20 out of 72 for the 2020-21 season
processing game 30 out of 72 for the 2020-21 season
processing game 40 out of 72 for the 2020-21 season
processing game 50 out of 72 for the 2020-21 season
processing game 60 out of 72 for the 2020-21 season
processing game 70 out of 72 for the 2020-21 season
2020 2021
------------------------

[{'homeAbrev': 'BKN',
  'homeId': 1610612751,
  'homeTeam': <__main__.Team at 0x1650fdab790>,
  'homeTeamPredictions': <__main__.Prediction at 0x16557e85c10>,
  'awayAbrev': 'NYK',
  'awayId': 1610612752,
  'awayTeam': <__main__.Team at 0x16557e85280>,
  'awayTeamPredictions': <__main__.Prediction at 0x1650e091cd0>},
 {'homeAbrev': 'TOR',
  'homeId': 1610612761,
  'homeTeam': <__main__.Team at 0x16557ed67f0>,
  'homeTeamPredictions': <__main__.Prediction at 0x1650fd98eb0>,
  'awayAbrev': 'MEM',
  'awayId': 1610612763,
  'awayTeam': <__main__.Team at 0x16557ed60d0>,
  'awayTeamPredictions': <__main__.Prediction at 0x16557ed6c10>},
 {'homeAbrev': 'PHX',
  'homeId': 1610612756,
  'homeTeam': <__main__.Team at 0x1650fdca2e0>,
  'homeTeamPredictions': <__main__.Prediction at 0x1650fdca850>,
  'awayAbrev': 'GSW',
  'awayId': 1610612744,
  'awayTeam': <__main__.Team at 0x1650fdca4f0>,
  'awayTeamPredictions': <__main__.Prediction at 0x1650fdcad90>},
 {'homeAbrev': 'POR',
  'homeId': 161061275

In [6]:
games = []
for i in range(len(teamPredData)):
    team1 = teamPredData[i]["homeTeam"]
    team2_abbreviation = teamPredData[i]["awayAbrev"]
    team1_prediction_class = teamPredData[i]["homeTeamPredictions"]
    team1_prediction_data = team1.get_data_for_prediction(team2_abbreviation)
    team1_prediction, team1_margin = team1_prediction_class.make_prediction(team1_prediction_data)

    team2 = teamPredData[i]["awayTeam"] 
    team1_abbreviation = teamPredData[i]["homeAbrev"]
    team2_prediction_class = teamPredData[i]["awayTeamPredictions"]
    team2_prediction_data = team2.get_data_for_prediction(team1_abbreviation)
    team2_prediction, team2_margin = team2_prediction_class.make_prediction(team2_prediction_data)
    
    team1ptavg = team1_prediction_data['Team Point Average']
    team1ptavg = round(team1ptavg.iloc[0],0)
    team2ptavg = team2_prediction_data['Team Point Average']
    team2ptavg = round(team2ptavg.iloc[0],0)

    game = pd.DataFrame([[team1_abbreviation,(round((team1_prediction.item(0)+team1_margin),0)),team2_abbreviation,(round((team2_prediction.item(0)+team2_margin),0)),datetime.today(), team1ptavg, team2ptavg]],columns=['Home','HomeScore','Away','AwayScore','Date','Home Point Avg','Away Point Avg'])
    games.append(game)
res = pd.concat(games)
res